In [1]:
!pip install bs4 re

  Obtaining dependency information for bs4 from https://files.pythonhosted.org/packages/51/bb/bf7aab772a159614954d84aa832c129624ba6c32faa559dfb200a534e50b/bs4-0.0.2-py2.py3-none-any.whl.metadata
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [6]:


# URL of the page to scrape
url = "https://disasters.geoplatform.gov/USA_Structures/"

# Send a GET request to fetch the HTML content of the page
response = requests.get(url)

# Save the HTML content to a local file
with open('USA_Structures.html', 'w', encoding='utf-8') as file:
    file.write(response.text)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all URLs that end with .zip (assuming these are the downloadable zip files)
zip_links = []
for link in soup.find_all('a', href=True):
    if re.search(r'\.zip$', link['href']):
        zip_links.append(link['href'])

# Print the list of zip file URLs
for zip_link in zip_links:
    print(zip_link)
    
print(len(zip_links))



https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Alabama/Deliverable20230526AL.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Alaska/Deliverable20230728AK.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/American+Samoa/Deliverable20230831AS.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Arizona/Deliverable20230502AZ.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Arkansas/Deliverable20230630AR.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/California/Deliverable20230728CA.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Colorado/Deliverable20230630CO.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Connecticut/Deliverable20230502CT.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structures/Delaware/Deliverable20230630DE.zip
https://fema-femadata.s3.amazonaws.com/Partners/ORNL/USA_Structure